OKX-hsyquant0 Trading Bot

[OKX API Documentation](https://www.okx.com/docs-v5/en/#overview)

In [1]:
import json
import okx.Account as Account
import okx.Trade as Trade

f_apikeys = 'apikeys.json'
apikeys = json.load(open(f_apikeys))

flag = "0" # live trading:0 , demo trading：1
api_key = apikeys['apikey']
secret_key = apikeys['secretkey']
passphrase = apikeys['passphrase']

accountAPI = Account.AccountAPI(api_key, secret_key, passphrase, False, flag)
tradeAPI = Trade.TradeAPI(api_key, secret_key, passphrase, False, flag)

Set Position Mode

In [ ]:
accountAPI.set_position_mode(
    posMode="long_short_mode"
)

accountAPI.set_leverage(
    instId = "BTC-USDT-SWAP",
    lever = "5",
    mgnMode = "cross"
)

accountAPI.set_leverage(
    instId = "ETH-USDT-SWAP",
    lever = "5",
    mgnMode = "cross"
)

# Check the balance

In [35]:
result = accountAPI.get_account_balance()
# print(json.dumps(result['data'], indent=2))

disEq=0
for curr in result['data'][0]['details']:
    disEq += float(curr['disEq'])
    print(f"ccy: {curr['ccy']}, cashBal: {curr['cashBal']}")

print(f"Total Value in USD: {disEq}")

body:  
header:  {'Content-Type': 'application/json', 'OK-ACCESS-KEY': '47947c28-9e11-475e-9f51-624f7e333e90', 'OK-ACCESS-SIGN': b'pdKasMCyfgulo6sEVvknHc+NE+smZMxCfuUgVMpg37k=', 'OK-ACCESS-TIMESTAMP': '2024-01-11T03:52:27.646Z', 'OK-ACCESS-PASSPHRASE': 'zyzzit-zEjqur-jijwo8', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/account/balance
ccy: USDT, cashBal: 2000
Total Value in USD: 2000.5799999999997


# Make an order

In [20]:
# limit order
tradeAPI.place_multiple_orders([
    {
        "instId": "BTC-USDT-SWAP",
        "tdMode": "cross",
        "side": "buy",
        "posSide": "long",
        "ordType": "limit",
        "px": "10",
        "sz": "1" # The BTC lot size is 0.01, so `sz` 5 means 0.05 BTC
    },
    {   
        "instId": "ETH-USDT-SWAP",
        "tdMode": "cross",
        "side": "sell",
        "posSide": "short",
        "ordType": "limit",
        "px": "10000",
        "sz": "1" # The ETH lot size is 0.1, so `sz` 2 means 0.2 ETH
    }
])

body:  [{"instId": "BTC-USDT-SWAP", "tdMode": "cross", "side": "buy", "posSide": "long", "ordType": "limit", "px": "10", "sz": "1"}, {"instId": "ETH-USDT-SWAP", "tdMode": "cross", "side": "sell", "posSide": "short", "ordType": "limit", "px": "10000", "sz": "1"}]
header:  {'Content-Type': 'application/json', 'OK-ACCESS-KEY': '47947c28-9e11-475e-9f51-624f7e333e90', 'OK-ACCESS-SIGN': b'WbvuQDjOFgotATX21Pb7DOZw5k8FHv9rZxqm0F4oFWo=', 'OK-ACCESS-TIMESTAMP': '2024-01-11T03:00:33.563Z', 'OK-ACCESS-PASSPHRASE': 'zyzzit-zEjqur-jijwo8', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/trade/batch-orders


{'code': '0',
 'data': [{'clOrdId': '',
   'ordId': '665510357548797957',
   'sCode': '0',
   'sMsg': 'Order placed',
   'tag': ''},
  {'clOrdId': '',
   'ordId': '665510357548797958',
   'sCode': '0',
   'sMsg': 'Order placed',
   'tag': ''}],
 'inTime': '1704942033824788',
 'msg': '',
 'outTime': '1704942033826087'}

# Orders Cancelled

In [36]:
order_list = tradeAPI.get_order_list()
orders = order_list['data']

for order in orders:
    tradeAPI.cancel_order(order['instId'], ordId=order['ordId'])

print("All orders cancelled")

body:  
header:  {'Content-Type': 'application/json', 'OK-ACCESS-KEY': '47947c28-9e11-475e-9f51-624f7e333e90', 'OK-ACCESS-SIGN': b'9x/9t9GdCxzFolh41F2h5/QdF9PGTGn7rbNFpwSjyhw=', 'OK-ACCESS-TIMESTAMP': '2024-01-11T03:54:57.079Z', 'OK-ACCESS-PASSPHRASE': 'zyzzit-zEjqur-jijwo8', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/trade/orders-pending
body:  {"instId": "ETH-USDT-SWAP", "ordId": "665510357548797958", "clOrdId": ""}
header:  {'Content-Type': 'application/json', 'OK-ACCESS-KEY': '47947c28-9e11-475e-9f51-624f7e333e90', 'OK-ACCESS-SIGN': b'Ja+6NlL8oxpOPCyd+MoEfLVPerJnjqMvvGZPdJ3tZaw=', 'OK-ACCESS-TIMESTAMP': '2024-01-11T03:54:58.130Z', 'OK-ACCESS-PASSPHRASE': 'zyzzit-zEjqur-jijwo8', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/trade/cancel-order
body:  {"instId": "BTC-USDT-SWAP", "ordId": "665510357548797957", "clOrdId": ""}
header:  {'Content-Type': 'application/json', 'OK-ACCESS-KEY': '47947c28-9e11-475e-9f51-624f7e333e90', 'OK-ACCESS-